In [ ]:
# Databricks notebook source
# ================================================================
# 🚀 PRODUCTION BATCH INFERENCE (XGBOOST ONLY)
# ================================================================

%pip install xgboost
dbutils.library.restartPython()

from databricks.sdk import WorkspaceClient
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np
from datetime import datetime
import math

print("=" * 80)
print("🚀 PRODUCTION BATCH INFERENCE (XGBOOST)")
print("=" * 80)

# =============================================================================
# CONFIGURATION
# =============================================================================
UC_CATALOG = "workspace"
UC_SCHEMA = "ml"
DATA_CATALOG = "workspace"
DATA_SCHEMA = "default"
INPUT_TABLE = "house_price_delta"

BATCH_SIZE = 100
OUTPUT_TABLE = f"{DATA_CATALOG}.{DATA_SCHEMA}.prod_inference_xgboost"
ENDPOINT_NAME = "house-price-xgboost-prod"

# =============================================================================
# INIT CLIENTS
# =============================================================================
try:
    w = WorkspaceClient()
    spark = SparkSession.builder.appName("XGBoostInference").getOrCreate()
    print("✓ Workspace & Spark initialized")
except Exception as e:
    dbutils.notebook.exit(f"INIT_FAILED: {e}")

# =============================================================================
# VERIFY ENDPOINT STATUS
# =============================================================================
print("\n🔍 Checking endpoint readiness...")
try:
    endpoint = w.serving_endpoints.get(name=ENDPOINT_NAME)
    if getattr(endpoint.state, "ready", False):
        print(f"✅ Endpoint '{ENDPOINT_NAME}' is READY")
    else:
        print(f"⚠️ Endpoint NOT fully ready → Proceeding cautiously")
except Exception as e:
    dbutils.notebook.exit(f"ENDPOINT_ERROR: {e}")

# =============================================================================
# LOAD INPUT DATA
# =============================================================================
print("\n📦 Loading input data...")
try:
    df_spark = spark.read.format("delta").table(f"{DATA_CATALOG}.{DATA_SCHEMA}.{INPUT_TABLE}")
    df = df_spark.toPandas()
    print(f"✅ Loaded {len(df)} records")
except Exception as e:
    dbutils.notebook.exit(f"DATA_LOAD_FAILED: {e}")

y_true = df["price"] if "price" in df.columns else None
FEATURE_COLUMNS = [c for c in df.columns if c not in ["price", "id", "timestamp"]]
print(f"🔍 Feature Columns: {FEATURE_COLUMNS}")

# =============================================================================
# BATCH INFERENCE
# =============================================================================
print("\n🚀 Performing inference...")
all_predictions = []
num_batches = (len(df) + BATCH_SIZE - 1) // BATCH_SIZE

try:
    for batch_idx in range(num_batches):
        start = batch_idx * BATCH_SIZE
        end = min((batch_idx + 1) * BATCH_SIZE, len(df))
        batch_df = df.iloc[start:end][FEATURE_COLUMNS]

        response = w.serving_endpoints.query(
            name=ENDPOINT_NAME,
            dataframe_records=batch_df.to_dict("records")
        )

        predictions = response.predictions
        all_predictions.extend(predictions)

        print(f"   → Batch {batch_idx+1}/{num_batches} complete")

except Exception as e:
    dbutils.notebook.exit(f"INFERENCE_FAILED: {e}")

df["predicted_price"] = all_predictions
df["prediction_timestamp"] = datetime.now()
df["endpoint_name"] = ENDPOINT_NAME
df["inference_method"] = "serving_endpoint"
print(f"✅ Generated {len(all_predictions)} predictions")

# =============================================================================
# SAVE RESULTS TO DELTA (AVOID DUPLICATES)
# =============================================================================
print("\n💾 Saving predictions...")
try:
    df["prediction_date"] = datetime.now().strftime('%Y-%m-%d')
    df["batch_id"] = datetime.now().strftime('%Y-%m-%d_%H%M%S')

    spark_df = spark.createDataFrame(df)

    # Duplicate check
    try:
        prev_df = spark.read.table(OUTPUT_TABLE).toPandas()
        if len(prev_df) > 0:
            last_pred_prev = prev_df["predicted_price"].iloc[-1]
            last_pred_new = df["predicted_price"].iloc[-1]
            if math.isclose(last_pred_prev, last_pred_new, rel_tol=1e-6):
                print("ℹ️ Duplicate batch detected — skipping save")
                dbutils.notebook.exit("SKIPPED_DUPLICATE")
    except:
        print("ℹ️ Output table does not exist → Creating new one")

    spark_df.write.mode("append").format("delta").option("mergeSchema", "true").saveAsTable(OUTPUT_TABLE)
    print(f"✅ Saved to {OUTPUT_TABLE}")

except Exception as e:
    dbutils.notebook.exit(f"SAVE_FAILED: {e}")

# =============================================================================
# PERFORMANCE METRICS
# =============================================================================
if y_true is not None and len(y_true) > 0:
    print("\n📊 Evaluating model performance...")
    y_pred = df["predicted_price"]

    mae = np.mean(np.abs(y_true - y_pred))
    rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))
    mape = np.mean(np.abs((y_true - y_pred) / np.where(y_true==0, 1, y_true))) * 100

    print(f"   MAE  : {mae:.3f}")
    print(f"   RMSE : {rmse:.3f}")
    print(f"   MAPE : {mape:.2f}%")

# =============================================================================
# SUMMARY
# =============================================================================
print("\n" + "="*80)
print("🎯 PRODUCTION INFERENCE COMPLETE")
print("="*80)
print(f"Model Type     : XGBOOST")
print(f"Endpoint Used  : {ENDPOINT_NAME}")
print(f"Output Table   : {OUTPUT_TABLE}")
print(f"Rows Processed : {len(df)}")
print(f"Timestamp      : {datetime.now()}")
print("="*80)

dbutils.notebook.exit("SUCCESS")
